In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import sklearn.pipeline as pipe

In [2]:
first = pd.read_csv('loc1.txt', sep=r'\s+')
first = first.drop('ind', axis=1)
second = pd.read_csv('loc2.txt', sep=r'\s+')
second = second.drop('ind', axis=1)
third = pd.read_csv('loc3.txt', sep=r'\s+')
third = third.drop('ind', axis=1)

In [3]:
#Небольшая процедура для предварительного анализа данных
def define_dataset(df):
      print(df.shape)
      print(df.info())
      print(df.head(40))
      print(df.describe())
      return df['id'].value_counts().head(20) #определение  индексов-дубликатов Функция get_dummies
#Формируем наборы признаков и вектор целевого признака для всех трех  локаций, одинакого исключая из списка признаков
#идентификатор (индекс) месторождения - он никак не может влиять на объем добытой нефти

features_1 = first.drop(['id','product'], axis=1)
features_ohe_1 = pd.get_dummies(features_1, drop_first=True)
target_1 = first['product']

features_2 = second.drop(['id','product'], axis=1)
features_ohe_2 = pd.get_dummies(features_2, drop_first=True)
target_2 = second['product']

features_3 = third.drop(['id','product'], axis=1)
features_ohe_3 = pd.get_dummies(features_3, drop_first=True)
target_3 = third['product']

#Разбиваем данные на обучающую и валидационную выборки в соотношении 75:25.
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_ohe_1, target_1, test_size=0.25, random_state=12345)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_ohe_2, target_2, test_size=0.25, random_state=12345)
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(features_ohe_3, target_3, test_size=0.25, random_state=12345)
model_1 = LinearRegression() #Применяем модель линейной регрессии
model_2 = LinearRegression()
model_3 = LinearRegression()

#Признаки кодируем во избежание доминирования одного из них
numeric = ['f0','f1','f2']
def scale(features_train, features_valid = None, numeric=['f0','f1','f2']):
      scaler = StandardScaler()
      scaler.fit(features_train_1[numeric])
      features_train[numeric] = scaler.transform(features_train[numeric])
      if features_valid is not None:
            features_valid[numeric] = scaler.transform(features_valid[numeric])
      return

#Обучаем модель и проводим предсказания на первой валидационной выборке.
def study(model: LinearRegression, features_train, features_valid, target_train, target_valid, number_location):
      model.fit(features_train,target_train) # обучите модель на первой тренировочной выборке
      predictions_valid = model.predict(features_valid) # получите  предсказания модели на первой валидационной выборке
      #Выводим на печать средний запас предсказанного сырья и RMSE модели для первой локации.
      mse = mean_squared_error(target_valid, predictions_valid)

      # < извлекаем корень из MSE >
      result =  mse ** 0.5
      print("Средний запас предсказанного на валидационной выборке", number_location, "сырья:", predictions_valid.mean(), '(тыс. баррелей)')
      print("RMSE модели линейной регрессии на валидационной выборке", number_location, ":", result)
      return predictions_valid

In [4]:
model_1 = pipe.Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])
study(model_1, features_train_1, features_valid_1, target_train_1, target_valid_1, 1)

Средний запас предсказанного на валидационной выборке 1 сырья: 101.57898111447821 (тыс. баррелей)
RMSE модели линейной регрессии на валидационной выборке 1 : 1.1414291393555058


array([135.66029436,  83.40132081, 135.85984776, 136.15562042,
        83.1414788 ,  55.74472549, 138.9568881 , 107.85486195,
         3.90662616, 135.10814729])

In [5]:
model_2 = pipe.Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])
study(model_2, features_train_2, features_valid_2, target_train_2, target_valid_2, 2)

Средний запас предсказанного на валидационной выборке 2 сырья: 79.60799856167405 (тыс. баррелей)
RMSE модели линейной регрессии на валидационной выборке 2 : 39.352398796307256


array([86.25531811, 94.42212813, 54.78362276, 78.82098344, 83.11798094,
       69.22219427, 85.62882546, 76.97265683, 69.07739358, 97.7788821 ])

In [6]:
model_3 = pipe.Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])
study(model_3, features_train_3, features_valid_3, target_train_3, target_valid_3, 3)

Средний запас предсказанного на валидационной выборке 3 сырья: 99.64955863171336 (тыс. баррелей)
RMSE модели линейной регрессии на валидационной выборке 3 : 45.50537343061165


array([121.36420993,  88.19996895,  90.82085875, 109.21275929,
        83.80523339, 105.59403231, 108.2624712 , 103.85781818,
        93.75486443,  91.62336988])

In [7]:
r1 = pd.read_csv('place1.csv', sep=',')
r2 = pd.read_csv('place2.csv', sep=',')
r3 = pd.read_csv('place3.csv', sep=',')

In [8]:
COSTS = 500_000 #бюджет на разработку
INCOME = 450 #доход с одного бареля нефти
COUNT_REGION = 30 #количество исследуемых точек в одном регионе
BOREHOLES = 16 #количество выбранных скважин для разработки  месторождения
loss_threshold = COSTS/(BOREHOLES*INCOME) #Минимальная средняя продуктивность скважины для достижения порога окупаемости
region_threshold = round(BOREHOLES*loss_threshold,1) #Минимальная продуктивность 200 скважин региона для достижения порога окупаемости
print('Минимальная средняя продуктивность скважины для достижения порога окупаемости:', round(loss_threshold,1), '(тыс. баррелей)')

Минимальная средняя продуктивность скважины для достижения порога окупаемости: 69.4 (тыс. баррелей)


In [9]:
def calc_profit(data: pd.DataFrame, model: LinearRegression):
  d = data.copy()
  # d.sample()
  d_product = model.predict(d[numeric])
  # print(d_product)
  d['product'] = d_product
  d.sort_values(by='product', inplace=True)
  top_d = d.tail(BOREHOLES)
  # print(top_d)
  return top_d['product'].sum() * INCOME - COSTS
  

In [10]:
print("Прибыль в первой локации:", calc_profit(r1, model_1).round(),'тыс рублей')
print("Прибыль в первой локации:", calc_profit(r2, model_2).round(),'тыс рублей')
print("Прибыль в первой локации:", calc_profit(r3, model_3).round(),'тыс рублей')

Прибыль в первой локации: 663327.0 тыс рублей
Прибыль в первой локации: 318284.0 тыс рублей
Прибыль в первой локации: 442428.0 тыс рублей


In [11]:
import numpy as np

In [12]:
state = np.random.RandomState(12345) #обеспечим случайность формируемых выборок

In [13]:
def bootstrapped(data: pd.DataFrame, model: LinearRegression):
  values = []
  d = data.copy()
  for _ in range(1000):
    profit = calc_profit(d.sample(COUNT_REGION, replace=False, random_state=state), model)
    values.append(profit.round())
  
  values = pd.Series(values)
  mean = values.mean() #расчет средней прибыли
  print('Средняя прибыль, тыс руб.: {:,.2f}'.format(mean))

  lower = values.quantile(.025) #строим доверительный интервал
  upper = values.quantile(.975)

  print('95% доверительный интервал:', '{:,.2f}'.format(lower), ':', '{:,.2f}'.format(upper))

In [17]:
bootstrapped(r1, model_1)

Средняя прибыль, тыс руб.: 511,942.53
95% доверительный интервал: 375,761.05 : 629,147.95


In [15]:
bootstrapped(r2, model_2)

Средняя прибыль, тыс руб.: 263,213.98
95% доверительный интервал: 211,803.25 : 307,432.00


In [16]:
bootstrapped(r3, model_3)

Средняя прибыль, тыс руб.: 405,355.43
95% доверительный интервал: 367,463.42 : 435,708.30
